# Lab Assignment 9 - Not Sorting, but Networks

We're not doing sorting anymore, but we're going to skip ahead to next week's content on graphs and networks. But we'll mix it up a bit with some web-scraping as well.

Import all the stuff we might need.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sb
import requests
import networkx as nx
from bs4 import BeautifulSoup, NavigableString

# Problem 1: Get the data

Get the tournament page data.

In [4]:
raw = requests.get('http://www.sports-reference.com/cbb/postseason/2017-ncaa.html').text
soup = BeautifulSoup(raw,'lxml')

Parse out all the divisions.

In [8]:
east_soup = soup.find_all('div',{'id':'east'})
midwest_soup = soup.find_all('div',{'id':'midwest'})
south_soup = soup.find_all('div',{'id':'south'})
west_soup = soup.find_all('div',{'id':'west'})

Write a function to get all the teams in each division by parsing the HTML.

In [21]:
east_soup[0].find_all('a')[8]['href']

'/cbb/schools/southern-methodist/2017.html'

['/cbb/schools/villanova/2017.html',
 '/cbb/schools/mount-st-marys/2017.html',
 '/cbb/schools/wisconsin/2017.html',
 '/cbb/schools/virginia-tech/2017.html',
 '/cbb/schools/virginia/2017.html',
 '/cbb/schools/north-carolina-wilmington/2017.html',
 '/cbb/schools/florida/2017.html',
 '/cbb/schools/east-tennessee-state/2017.html',
 '/cbb/schools/southern-methodist/2017.html',
 '/cbb/schools/baylor/2017.html',
 '/cbb/schools/new-mexico-state/2017.html',
 '/cbb/schools/south-carolina/2017.html',
 '/cbb/schools/marquette/2017.html',
 '/cbb/schools/duke/2017.html',
 '/cbb/schools/troy/2017.html']

In [36]:
def get_teams(division_soup):
    teams_href_list = list()
    for link in division_soup[0].find_all('a'):
        if len(link.text) > 0:
            teams_href_list.append(link['href'])

    return teams_href_list

Run the function on each division's `soup` and add them all together for a list of all teams in the 2017 tournament.

In [38]:
east_teams_2017 = get_teams(east_soup)
midwest_teams_2017 = get_teams(midwest_soup)
south_teams_2017 = get_teams(south_soup)
west_teams_2017 = get_teams(west_soup)
teams_2017 = east_teams_2017 + midwest_teams_2017 + south_teams_2017 + west_teams_2017
len(teams_2017)

62

Make a `tournament_teams` by looping through `teams_2017` and for each element applying the `.text` attribute to extract the cleaned name of the team and append it to `tournament_teams`.

In [17]:
tournament_teams = 

Get the `href` out of a single team in `teams_2017`.

In [7]:
teams_2017[0]['href']

'/cbb/schools/villanova/2017.html'

We actually want the "2017-schedule.html" not "2017.html" Use the `replace` string function to update the `href`.

In [46]:
'http://www.sports-reference.com' + teams_2017[0].replace('2017','2017-schedule')

'http://www.sports-reference.com/cbb/schools/villanova/2017-schedule.html'

Crawl the data for a single team:

1. Make a `_url` variable that combines the domain ("http://www.sports-reference.com/") and the `href` updated to contain "2017-schedule.html".
2. Use requests' `get` and `text` methods to get the raw HTML, saving it as `team_raw`
3. Make `team_soup` out of `team_raw`
4. Find the specific table containing the schedule and save it as `schedule_table`.

**HINT**: The table has both a "class" and an "id", both of which you can pass to the `.find_all()` method inside a dictionary.

In [65]:
raw = requests.get('http://www.sports-reference.com/cbb/schools/villanova/2017-schedule.html').text
soup = BeautifulSoup(raw,'lxml')

schedule_table = soup.find_all('table',{'id':'schedule','class':"sortable stats_table"})
row = schedule_table[0].find_all('tr')[10]
row.find_all('td')

[<td class="left " csk="2016-12-10" data-stat="date_game"><a href="/cbb/boxscores/2016-12-10-notre-dame.html">Sat, Dec 10, 2016</a></td>,
 <td class="left " data-stat="time_game">12:00 pm/est</td>,
 <td class="left " data-stat="network"></td>,
 <td class="left " data-stat="game_type">REG</td>,
 <td class="left " data-stat="game_location">N</td>,
 <td class="left " data-stat="opp_name"><a href="/cbb/schools/notre-dame/2017.html">Notre Dame</a> <span class="note">(23)</span></td>,
 <td class="left " data-stat="conf_abbr"><a href="/cbb/conferences/acc/2017.html" title="Atlantic Coast Conference">ACC</a></td>,
 <td class="left " data-stat="game_result">W</td>,
 <td class="right " data-stat="pts">74</td>,
 <td class="right " data-stat="opp_pts">66</td>,
 <td class="center " data-stat="overtimes"></td>,
 <td class="right " data-stat="wins">10</td>,
 <td class="right " data-stat="losses">0</td>,
 <td class="left " csk="10" data-stat="game_streak">W 10</td>,
 <td class="left " data-stat="arena

In [71]:
schedule_table[0].find_all('td',{'data-stat':'opp_pts'})[1].text

'76'

# Problem 2: Parse the `schedule_table`

Using the `schedule_table` soup object, parse out all of the relevant cells.

First, parse out the dates and save this list of values as `dates`. Your best bet is to take advantage of the "data-stat" tag identifer for each of the steps below. 

Second, parse out the opponents and save this list of values as `opponents`.

Third, parse out the results and save this list of values as `results`.

Fourth, parse out the team score and save this list of values as `team_scores`.

Fifth, parse out the opponent's score as save this list of values as `opp_scores`.

Before the final step, a brief aside on how Python's zip function works. Consider three lists below, the `zip` function will combine them all for you. Note that `zip` returns an iterator.

In [8]:
list1 = ['a','b','c']
list2 = [0,1,2]
list3 = ['alpha','beta','gamma']

zip(list1,list2,list3)

So we wrap the `zip` inside a `list` to make it spill its guts. We see that the first element of `list1`, the first element of `list2`, and the first element of `list3` were combined together into a tuple and this tuple is the first element in a list. Then the second elements of the lists became a tuple in the second elemnt of the list, *etc*.

In [9]:
list(zip(list1,list2,list3))

[('a', 0, 'alpha'), ('b', 1, 'beta'), ('c', 2, 'gamma')]

Finally, combine all five of the lists of values you extracted above together using Python's `zip` function and save the list of results as `team_results`. Show your team results by calling `team_results`.

# Problem 3: Crawl all of the teams' tables

Write a loop that goes through each team in `teams_2017` and does all of the steps above in each pass.

0. Before the loop starts, has an empty list `all_results`
1. Checks to make sure the `team.text` isn't empty, skips urls that point to "tbd", or handles index position errors from pages with empty tables.
2. Takes the team object the url with the `href` replacement
3. Uses requests' `get` and `text` methods to get the raw HTML
4. Turns the raw HTML into Soup
5. Finds the table
6. Finds the dates, opponents, results, team score, and opponent scores
7. Zips them all together into `team_results`
8. For each result in `team_results` appends it to `all_results`

There's 64 team pages to visit, so if it takes ~0.5 seconds to download and parse each one, this might take about 30 seconds to run.

In [13]:
all_results = list()
for team in teams_2017:
    if len(team.text) > 0:
        

Loop over `all_results` and only keep results for winners to avoid double-counting by adding them to a new `reduced_results` list. You can identify winners either by the result tag or by comparing the score and opp_score values.

Print out the length of the `all_results` and `reduced_results` to confirm that you really got rid of the games. How many games remain?

In [20]:
reduced_results = list()

        
len(all_results), len(reduced_results)

(2060, 1455)

Now reduce the results again to only include the matches between teams that started out in the 1st round of the tournament. Loop through `reduced_results`, check if the opponent is in `tournament_teams` (defined in Problem 1), and add the result to `tournament_results`.

In [25]:
tournament_results = 

[('Villanova', 'Purdue', '2016-11-14', '79', '76'),
 ('Villanova', 'Notre Dame', '2016-12-10', '74', '66'),
 ('Villanova', 'Creighton', '2016-12-31', '80', '70'),
 ('Villanova', 'Marquette', '2017-01-07', '93', '81'),
 ('Villanova', 'Xavier', '2017-01-10', '79', '54'),
 ('Villanova', 'Seton Hall', '2017-01-16', '76', '46'),
 ('Villanova', 'Virginia', '2017-01-29', '61', '59'),
 ('Villanova', 'Xavier', '2017-02-11', '73', '57'),
 ('Villanova', 'Seton Hall', '2017-02-18', '92', '70'),
 ('Villanova', 'Creighton', '2017-02-25', '79', '63')]

At this stage, your `tournament_results` should look something like this:

# To do in class: Make a directed graph

If team *i* beat team *j* by *w* points, create an edge from node *i* towards node *j* with weight *w*.

In [28]:
g = nx.DiGraph()

for (team, opponent, date, score, opp_score) in tournament_results:
    differential = int(score) - int(opp_score)
    if differential > 0:
        if g.has_edge(team,opponent):
            g[team][opponent]['weight'] += differential
        else:
            g.add_edge(team, opponent, weight = abs(differential))
    else:
        print('Negative point differential for: {0}-{1}\n'.format(team,opponent))
        if g.has_edge(team,opponent):
            g[team][opponent]['weight'] += differential
        else:
            g.add_edge(team, opponent, weight = abs(differential))
        
print("There are {0} nodes and {1} edges in the network".format(g.number_of_nodes(), g.number_of_edges()))

nx.write_gexf(g,'tournament_schedule.gexf')

Negative point differential for: Creighton-Marquette

Negative point differential for: Michigan-UCLA

Negative point differential for: Texas Southern-Louisville

Negative point differential for: Winthrop-Florida State

There are 58 nodes and 222 edges in the network
